## Import

Load data.

In [1]:
import os
import pandas as pd
from datetime import date
from utils import *
from unicodedata import normalize

save = True
delim = "|"
timestamp = date.today().strftime("%Y%m%d")
in_path = os.path.join("input", "single_char_20230616.csv")
out_path = os.path.join("output", f"manual_input_{timestamp}.csv")
audio_dir = os.path.join("output", "audio")

cols = ["Simp", "Trad", "Meaning", "Simp (Color)", "Trad (Color)", "Pinyin (Color)", "Sound", "Silhouette"]
df = pd.read_csv(in_path, dtype = str)
df.columns = ["Simp", "Trad", "Pinyin (Color)", "Meaning"]
df = df[[col for col in cols if col in df.columns]]
df_edited = df.copy()

print("Shape:", df.shape)
df.head()

Shape: (92, 4)


,Simp,Trad,Meaning,Pinyin (Color)
0,想,想,to think; to believe; to wish; to want; to mis...,xiǎng
1,从,从,from; through,cóng
2,只,只,only; just; but,zhǐ
3,因,囙,cause; reason,yīn
4,但,但,but; yet; however,dàn


# Basic Formatting

In [2]:
df_edited["Meaning"] = df["Meaning"].apply(lambda x : x.replace("; ", "<br>"))
df_edited["Silhouette"] = df["Simp"].astype(str).apply(get_silhouette)
df_edited = df_edited[[col for col in cols if col in df_edited.columns]]

print("\nView table:")
df_edited.head()


View table:


,Simp,Trad,Meaning,Pinyin (Color),Silhouette
0,想,想,to think<br>to believe<br>to wish<br>to want<b...,xiǎng,_
1,从,从,from<br>through,cóng,_
2,只,只,only<br>just<br>but,zhǐ,_
3,因,囙,cause<br>reason,yīn,_
4,但,但,but<br>yet<br>however,dàn,_


## Coloring

Convert characters to pinyin. Then add color.

In [3]:
pin_split = (
    df_edited["Pinyin (Color)"]
        .apply(lambda x : normalize("NFC", x))
        .apply(PINYIN_RE.split)
        .apply(delim.join)
        .apply(full_split)
)
simp_split = df_edited["Simp"].apply(delim.join).apply(full_split, args = delim)
trad_split = df_edited["Trad"].apply(delim.join).apply(full_split, args = delim)

df_edited["Simp (Color)"] = pd.concat([pin_split, simp_split], axis = 1).apply(lambda row: add_color_hanzi(row[0], row[1]), axis = 1)
df_edited["Trad (Color)"] = pd.concat([pin_split, trad_split], axis = 1).apply(lambda row: add_color_hanzi(row[0], row[1]), axis = 1)
df_edited["Pinyin (Color)"] = pin_split.apply(simplify_split).apply(tag_split)

df_edited = df_edited[[col for col in cols if col in df_edited.columns]]
df_edited.head()

,Simp,Trad,Meaning,Simp (Color),Trad (Color),Pinyin (Color),Silhouette
0,想,想,to think<br>to believe<br>to wish<br>to want<b...,<span class = 'tone3'>想</span>,<span class = 'tone3'>想</span>,<span class = 'tone3'>xiǎng</span>,_
1,从,从,from<br>through,<span class = 'tone2'>从</span>,<span class = 'tone2'>从</span>,<span class = 'tone2'>cóng</span>,_
2,只,只,only<br>just<br>but,<span class = 'tone3'>只</span>,<span class = 'tone3'>只</span>,<span class = 'tone3'>zhǐ</span>,_
3,因,囙,cause<br>reason,<span class = 'tone1'>因</span>,<span class = 'tone1'>囙</span>,<span class = 'tone1'>yīn</span>,_
4,但,但,but<br>yet<br>however,<span class = 'tone4'>但</span>,<span class = 'tone4'>但</span>,<span class = 'tone4'>dàn</span>,_


In [4]:
df_edited[df_edited["Simp (Color)"] == "(color error)"]

,Simp,Trad,Meaning,Simp (Color),Trad (Color),Pinyin (Color),Silhouette


# Sound

In [5]:
df_edited["Sound"] = df_edited["Simp"].apply(lambda s : save_audio(s, audio_dir, save))
df_edited = df_edited[[col for col in cols if col in df_edited.columns]]

errors = df_edited["Sound"][df_edited["Sound"].apply(lambda s : s.startswith("FAILED"))]
print("Errors:\n", errors)
df_edited.head()

Errors:
 Series([], Name: Sound, dtype: object)


,Simp,Trad,Meaning,Simp (Color),Trad (Color),Pinyin (Color),Sound,Silhouette
0,想,想,to think<br>to believe<br>to wish<br>to want<b...,<span class = 'tone3'>想</span>,<span class = 'tone3'>想</span>,<span class = 'tone3'>xiǎng</span>,[sound:想.mp3],_
1,从,从,from<br>through,<span class = 'tone2'>从</span>,<span class = 'tone2'>从</span>,<span class = 'tone2'>cóng</span>,[sound:从.mp3],_
2,只,只,only<br>just<br>but,<span class = 'tone3'>只</span>,<span class = 'tone3'>只</span>,<span class = 'tone3'>zhǐ</span>,[sound:只.mp3],_
3,因,囙,cause<br>reason,<span class = 'tone1'>因</span>,<span class = 'tone1'>囙</span>,<span class = 'tone1'>yīn</span>,[sound:因.mp3],_
4,但,但,but<br>yet<br>however,<span class = 'tone4'>但</span>,<span class = 'tone4'>但</span>,<span class = 'tone4'>dàn</span>,[sound:但.mp3],_


# Save

In [6]:
if save:
    df_edited.to_csv(out_path, index = False, header = False)